In [1]:
from neo4j.graph import NeoGraph

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/philgooch/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
neo = NeoGraph('resources/scandal.txt')

In [3]:
neo.add_paradigmatic_relations()

0.19999999999999998